# Genic Notation

According to the paper, they have mapped to the human reference genome (hg19).
HG19 is the alias to GRCh37.

Human Gene Annotation - GENCODE Release 19 (GRCh37.p13)

https://www.gencodegenes.org/human/release_19.html

In [1]:
# Imports
import gzip
import shutil
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gffutils
import os

In [2]:
# File_in contains the comprehensive gene annotation on the reference chromosomes only
file_in = 'Data/ComprehensiveGeneAnnotation/gencode.v19.annotation.gff3.gz'
file_out = 'Data/ComprehensiveGeneAnnotation/gencode.v19.annotation.gff3'
db_file = 'Data/ComprehensiveGeneAnnotation/db_GRCh37.db'

# # File_in contains the basic gene annotation on the reference chromosomes only
# # This is a subset of the corresponding comprehensive annotation, including only those transcripts tagged as 'basic' in every gene
# # This is the main annotation file for most users
# file_in = 'gencode.v42.basic.annotation.gff3.gz'
# file_out = 'gencode.v42.basic.annotation.gff3'

with gzip.open(file_in, 'rb') as f_in:
    with open(file_out, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [3]:
if os.path.exists(db_file):
    db = gffutils.FeatureDB(dbfn=db_file)    
else:
    db = gffutils.create_db(file_out, ":memory:", merge_strategy='create_unique', id_spec={'gene': 'gene_id', 'transcript': 'transcript_id'},
    verbose = True)
    

2023-01-17 00:14:25,054 - INFO - Populating features
2023-01-17 00:20:45,180 - INFO - Populating features table and first-order relations: 2615565 features
2023-01-17 00:20:45,181 - INFO - Updating relations
2023-01-17 00:21:23,114 - INFO - Creating relations(parent) index
2023-01-17 00:21:25,983 - INFO - Creating relations(child) index
2023-01-17 00:21:29,364 - INFO - Creating features(featuretype) index
2023-01-17 00:21:31,264 - INFO - Creating features (seqid, start, end) index
2023-01-17 00:21:33,398 - INFO - Creating features (seqid, start, end, strand) index
2023-01-17 00:21:35,615 - INFO - Running ANALYZE features


In [4]:
def get_length_gen(gene_id, method='max'):
    """
    Calculates the length of the gene
    :param gene_id: id of gene, ex 'ENSG00000223972'
    :param method:method of length calculation [min, max, mean]
    """ 
    transcript_count = 0
    summation = 0
    result = 0
    c = db.conn.cursor()
    c.execute(f"SELECT id FROM features WHERE id like '{gene_id}.%'")
    id_version = c.fetchone()[0]
    for child_transcript in db.children(id_version, 1, featuretype='transcript'):
        transcript_count += 1
        axon_sum = sum(exon.stop - exon.start for exon in db.children(child_transcript.id, 1, featuretype='exon'))
        match method:
            case 'max':
                result = result if result > axon_sum else axon_sum
            case 'min':
                if result == 0:
                    result = axon_sum
                else:
                    result = result if result < axon_sum else axon_sum
            case 'mean':
                summation += axon_sum
                result = (summation)/transcript_count
    return result

In [37]:
def get_length_gen2(gene_id, method='max'):
    """
    Calculates the length of the gene
    :param gene_id: id of gene, ex 'ENSG00000223972'
    :param method:method of length calculation [min, max, mean]
    """ 
    transcript_count = 0
    summation = 0
    result = 0
    gid='ENSG00000223972'
    c = db.conn.cursor()
    c.execute(f"SELECT id FROM features WHERE id like '{gid}.%'")
    results = c.fetchone()[0]
    print(str(results))

In [5]:
g_length = get_length_gen('ENSG00000223972.4', method='mean')
print(g_length)

TypeError: 'NoneType' object is not subscriptable